<a href="https://colab.research.google.com/github/Structurebiology-BNL/CryoFEM/blob/main/Colab_CryoFEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CryoFEM (Cryo-EM Feature Enhancement Model)
CryoFEM is an image enhancement tool designed for cryo-EM density map post-processing. Utilizing 3D convolutional neural networks, it effectively enhances the quality of density maps, facilitating more precise interpretation at the atomic level. For an in-depth look at how CryoFEM achieves this, you can check out our [preprint](https://www.biorxiv.org/content/10.1101/2023.02.02.526877).

In this Colab notebook, we offer an easy-to-use instance of CryoFEM. You can upload your own half-maps, run CryoFEM to post-processing, and then download the improved map. 

In [ ]:
# @title Upload your half maps or using half maps from EMDB: {display-mode: "form"}
# #@markdown - You will be first asked to mount the Google drive with your google account.
from google.colab import files

# @markdown - Upload could take a while depending on the size of the map
# @markdown - You can upload two half maps at once or one half map each time
# @markdown - The output will be `{job_name}.mrc`, you can overwrite the default `jobname` below
# @markdown - After starting the session, make sure to change the `runtime type` in the `Runtime` menu to use GPU for faster performance.
import os


def check_file_existence(file_path):
    return os.path.isfile(file_path + ".mrc")


def get_available_jobname(jobname):
    if not check_file_existence(jobname):
        return jobname

    n = 0
    while check_file_existence(f"{jobname}_{n}"):
        n += 1

    return f"{jobname}_{n}"


jobname = "CryoFEM_enhanced"  # @param {type:"string"}
jobname = get_available_jobname(jobname)

half_map_1, half_map_2 = None, None

# in case the user has already uploaded the half maps and hit the `run all` button, we don't want to ask them to upload again
if "uploaded_map" in globals():
    try:
        if isinstance(uploaded_map, list) and len(uploaded_map) == 2:
            pass
    except NameError:
        uploaded_map = None
else:
    uploaded_map = []
    half_maps = {}
    while len(uploaded_map) != 2:
        if len(uploaded_map) == 0:
            print("Please upload two half maps.")
        elif len(uploaded_map) == 1:
            print("Please upload the second half map.")
        else:
            print("You have uploaded too many maps. Please upload exactly 2 maps.")
            uploaded_map = []
        half_maps = files.upload()
        uploaded_map += list(half_maps.keys())

    print("Two half maps are uploaded successfully.")

# #@markdown - Alternatively, you can choose to download half-maps from EMDB instead. {display-mode: "form"}
# #@markdown  - Make sure that the corresponding EMDB entry has half maps
# EMDB_ID = 23274  #@param {type: "number"}

In [30]:
# @title Package Installation {display-mode: "form"}
# @markdown Run this cell and Colab will:
# @markdown - Install necessary python packages
# @markdown - Download CryoFEM model
# @markdown - Preprocess the uploaded maps
# %%capture --no-stderr --no-display
!pip install -q mrcfile torchio
!pip install -q torch -f https://download.pytorch.org/whl/cu116/torch_stable.html
print("Package installation finished")
import os

if os.path.exists("/content/CryoFEM"):
    pass  # skip if already downloaded
else:
    print("Install CryoFEM")
    !git clone https://github.com/Structurebiology-BNL/CryoFEM
    %cd /content/CryoFEM
    !wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1aA12Hxu_1DAxZ5HvkncdZsB6p7BQU6S5&confirm=t' -O model_weights.pt
    print("CryoFEM download finished")
    import torch
    from tqdm import tqdm
    import mrcfile

    from models.map_splitter import reconstruct_maps, map_resample
    from models.unet import UNetRes

    if uploaded_map:
        file_path_1 = "/content/" + uploaded_map[0]
        if len(uploaded_map) == 1:
            print(
                "only one half map is uploaded, will proceed to enhance a single half map"
            )
            with mrcfile.open(file_path_1, mode="r") as input_map:
                _, averaged_map, input_data, meta_data = map_resample(
                    input_map, box_size=128, core_size=100
                )
        elif len(uploaded_map) == 2:
            file_path_2 = "/content/" + uploaded_map[1]
            with mrcfile.open(file_path_1, mode="r") as half_map_1, mrcfile.open(
                file_path_2, mode="r"
            ) as half_map_2:
                _, averaged_map, input_data, meta_data = map_resample(
                    half_map_1, input_map_2=half_map_2, box_size=128, core_size=100
                )
        else:
            print("no valid half maps are provided")
    # else:
    #     from utils.utils import download_half_maps
    #     assert isinstance(int(EMDB_ID), int), "EMDB ID must be an integer"
    #     download_succesful = download_half_maps(EMDB_ID)
    #     if download_succesful:
    #         print("Read downloaded map EMDB-{} and resample".format(EMDB_ID))
    #         input_map = mrcfile.open("averaged_map_{}.ccp4".format(EMDB_ID), mode="r")
    #         resampled_map, averaged_map, input_data, meta_data = map_resample(input_map)
    #     else:
    #         print("No valid half maps found for EMDB-{}".format(EMDB_ID))
    print("\nInput data ready")

In [ ]:
# @title Run prediction {display-mode: "form"}
# @markdown - The prediction should be done within a few minutes if using GPU
%cd /content/CryoFEM
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
model = UNetRes(n_blocks=2, act_mode="R")
checkpoint = torch.load("/content/CryoFEM/model_weights.pt", map_location="cpu")
model.load_state_dict(checkpoint)
model = model.to(device)

batch_size = 2
torch.backends.cudnn.benchmark = True
model.eval()
with torch.no_grad():
    y_pred = torch.tensor(())
    print("Start prediction...")
    for indx in tqdm(range(0, input_data.shape[0], batch_size)):
        x_partial = input_data[indx : indx + batch_size].unsqueeze(dim=1).to(device)
        y_pred_partial = model(x_partial)
        y_pred = torch.cat(
            (y_pred, y_pred_partial.squeeze(dim=1).detach().cpu()),
            dim=0,
        )
    original_shape = (
        int(meta_data.cella.x),
        int(meta_data.cella.y),
        int(meta_data.cella.z),
    )
    y_pred_recon = reconstruct_maps(
        y_pred.numpy(), original_shape, box_size=128, core_size=100
    )
    with mrcfile.new("{}.mrc".format(jobname), overwrite=True) as mrc:
        mrc.set_data(y_pred_recon)
        mrc.header.cella.x = meta_data.cella.x
        mrc.header.cella.y = meta_data.cella.y
        mrc.header.cella.z = meta_data.cella.z
        mrc.header.nxstart = meta_data.nxstart
        mrc.header.nystart = meta_data.nystart
        mrc.header.nzstart = meta_data.nzstart
    print("\nPrediction done!")

In [ ]:
# @title Visualization and download {display-mode: "form"}
# @markdown - Upper rows are the raw map (average of two half maps)
# @markdown - Lower rows are the enhanced map
# @markdown - Please wait for a while for the pop up window to show up to complete download
import torchio

averaged_map = torchio.ScalarImage(
    tensor=torch.from_numpy(averaged_map).unsqueeze(dim=0)
)
averaged_map.plot(
    figsize=(12, 9),
    radiological=False,
)
pred_map = torchio.ScalarImage(tensor=torch.from_numpy(y_pred_recon).unsqueeze(dim=0))
pred_map.plot(
    figsize=(12, 9),
    radiological=False,
)

files.download("/content/CryoFEM/{}.mrc".format(jobname))